In [1]:
import tensorflow as tf
import keras.backend as K
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from Project.utilities import *
from Project.dataStructures import *
from keras.models import Sequential
from keras.layers import PReLU, Dense, Activation,BatchNormalization,Dropout
import numpy as np

Using TensorFlow backend.


In [2]:
K.set_learning_phase(0)
rdm = np.random.RandomState()
rdm.seed(42)


In [3]:
import pandas as pd
data = pd.read_csv('/Users/laneskij/Documents/University/Tesi/Project/dataset.csv')
dataset = np.array(data)
rdm.shuffle(dataset)


In [8]:
X =dataset[:,:-1].astype(np.float32)
Y = dataset[:,-1]
label_dic = {target:value for value, target in enumerate(sorted(set(Y.tolist())))}
Y = [label_dic[y] for y in Y]

In [3]:
datasetList, attributes, coloumns = read_file('/dataset.csv')
targetsDic, targets = get_classes(coloumns['class'])
dataset = string_to_float(switch_label(datasetList, targets))
rdm.shuffle(dataset)
labels = []
for item in dataset:
    labels.append(item[-1])  
stringLabels = [targets[item-1] for item in labels]


In [4]:
X = np.array(dataset)[:,:len(dataset[0])-1].astype(np.float64)
Y = np.array(dataset)[:,len(dataset[0])-1].astype(np.int64)
Y = np.array([item-1 for item in Y])
xTrain, xTest = X[:int(999*0.8)], X[int(999*0.8):]
yTrain, yTest = Y[:int(999*0.8)], Y[int(999*0.8):]

In [5]:
scaler = MinMaxScaler(feature_range=(0,1))
X = scaler.fit_transform(X)
xTrain = scaler.fit_transform(xTrain)
xTest = scaler.transform(xTest)



In [6]:
xTrain, xTest = X[:int(1000*0.8)], X[int(1000*0.8):]
yTrain, yTest = Y[:int(1000*0.8)], Y[int(1000*0.8):]


In [7]:
layers = [
    Dense(50, input_dim=len(xTrain[0]),init='he_normal'), PReLU(), Dropout(0.2),BatchNormalization(), 
    Dense(10, init='he_uniform'), Activation('softmax')
]

In [8]:
model = Sequential()
for layer in layers:
    model.add(layer)


In [9]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adadelta')

In [ ]:

for e in range(100):
    losses = []
    for x_batch,y_batch in get_minibatches(xTrain,yTrain):
        loss = model.train_on_batch(x_batch,y_batch)
        losses.append(loss)
    
    train_acc = 0.
    for x_batch,y_batch in get_minibatches(xTrain,yTrain):
        train_acc += (np.argmax(model.predict(x_batch), axis=1) == y_batch[:,0]).sum()
    test_acc = 0.
    for x_batch,y_batch in get_minibatches(xTest,yTest):
        test_acc += (np.argmax(model.predict(x_batch), axis=1) == y_batch[:,0]).sum()
        
    #print 'Loss: %.3f - Train Acc: %.3f - Test Acc: %.3f' % (np.mean(losses), 100.*train_acc/len(yTrain),100.*test_acc/len(yTest))

In [11]:
layers = [
    Dense(50, input_dim=len(X[0]),init='he_normal'), PReLU(), Dropout(0.2), #BatchNormalization(), 
    Dense(20, init='he_normal'), PReLU(), Dropout(0.2),
    Dense(50, init='he_normal'), PReLU(),
    Dense(len(X[0]), 'he_uniform'),Activation('sigmoid')
]

In [12]:
ae = Sequential()
for layer in layers:
    ae.add(layer)

In [13]:
ae.compile(loss='binary_crossentropy', optimizer='adadelta')

In [15]:
for e in range(100):
    losses = []
    for xBatch,yBatch in get_minibatches(X,Y):
        loss = ae.train_on_batch(xBatch,xBatch)
        losses.append(loss)
    #print loss

In [16]:
print loss

0.535961


In [17]:
embedding = K.function([ae.layers[0].input, 0], [ae.layers[3].output])

In [18]:
newX = []
stringLabels = [targets[item-1] for item in labels]
outputX = embedding([X])[0]
for item in outputX:
    newX.append(item)
dataset = np.c_[np.array(newX), np.array(stringLabels)].tolist() 

    

In [19]:
outputX = embedding([X])[0]


In [20]:
import csv
with open('10' + '.csv', 'wb') as f:
    writer = csv.writer(f)
    writer.writerows(outputX)
